In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, GlobalAvgPool2D, Dense, Multiply, Reshape
from tensorflow.keras.optimizers import Adam
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import mean_squared_error as mse
from skimage.metrics import structural_similarity as ssim
from ipywidgets import interact, IntSlider  # For interactive comparison
import pandas as pd  # For quality report
import seaborn as sns  # For better visualizations

def generate_quality_report(image_names, all_scores):
    """Generate comprehensive quality report"""
    # Create DataFrame for metrics
    metrics_df = pd.DataFrame({
        'Image': image_names,
        'PSNR_Degraded': [s[0][0] for s in all_scores],
        'PSNR_Enhanced': [s[1][0] for s in all_scores],
        'MSE_Degraded': [s[0][1] for s in all_scores],
        'MSE_Enhanced': [s[1][1] for s in all_scores],
        'SSIM_Degraded': [s[0][2] for s in all_scores],
        'SSIM_Enhanced': [s[1][2] for s in all_scores]
    })
    
    # Calculate improvements
    metrics_df['PSNR_Improvement'] = metrics_df['PSNR_Enhanced'] - metrics_df['PSNR_Degraded']
    metrics_df['MSE_Reduction'] = metrics_df['MSE_Degraded'] - metrics_df['MSE_Enhanced']
    metrics_df['SSIM_Improvement'] = metrics_df['SSIM_Enhanced'] - metrics_df['SSIM_Degraded']
    
    # Save to CSV
    metrics_df.to_csv('output/quality_metrics.csv', index=False)
    
    # Create visualization
    plt.figure(figsize=(15, 8))
    plt.subplot(231)
    sns.barplot(x='Image', y='PSNR_Improvement', data=metrics_df)
    plt.title('PSNR Improvement')
    plt.xticks(rotation=45)
    
    plt.subplot(232)
    sns.barplot(x='Image', y='MSE_Reduction', data=metrics_df)
    plt.title('MSE Reduction')
    plt.xticks(rotation=45)
    
    plt.subplot(233)
    sns.barplot(x='Image', y='SSIM_Improvement', data=metrics_df)
    plt.title('SSIM Improvement')
    plt.xticks(rotation=45)
    
    plt.tight_layout()
    plt.savefig('output/quality_report.png', dpi=300, bbox_inches='tight')
    plt.close()
    
    return metrics_df

def interactive_comparison(original, degraded, enhanced):
    """Create interactive image comparison"""
    def display_images(alpha=0.5):
        plt.figure(figsize=(10, 5))
        plt.subplot(131)
        plt.imshow(cv2.cvtColor(original, cv2.COLOR_BGR2RGB))
        plt.title('Original')
        
        plt.subplot(132)
        plt.imshow(cv2.cvtColor(degraded, cv2.COLOR_BGR2RGB))
        plt.title('Degraded')
        
        plt.subplot(133)
        blended = cv2.addWeighted(degraded, 1-alpha, enhanced, alpha, 0)
        plt.imshow(cv2.cvtColor(blended, cv2.COLOR_BGR2RGB))
        plt.title(f'Enhanced (Blend: {alpha:.1f})')
        
        plt.tight_layout()
        plt.show()
    
    interact(display_images, alpha=IntSlider(min=0, max=10, step=1, value=5))

def compare_architectures(image_path):
    """Compare different model architectures"""
    architectures = {
        'SRCNN': build_model(with_attention=False),
        'SRCNN+Attention': build_model(with_attention=True)
        # Add more architectures as needed
    }
    
    results = {}
    for name, model in architectures.items():
        model.load_weights('3051crop_weight_200.h5')
        _, _, output, scores = predict(image_path, use_attention='Attention' in name)
        results[name] = {
            'output': output,
            'scores': scores[1]  # Reconstructed scores only
        }
    
    # Display comparison
    print("\nArchitecture Comparison:")
    print("{:<20} {:>10} {:>12} {:>12}".format('Model', 'PSNR (dB)', 'MSE', 'SSIM'))
    print("-" * 50)
    for name, result in results.items():
        print("{:<20} {:>10.2f} {:>12.2f} {:>12.4f}".format(name, *result['scores']))
    
    # Visual comparison
    plt.figure(figsize=(15, 5))
    for i, (name, result) in enumerate(results.items(), 1):
        plt.subplot(1, len(architectures), i)
        plt.imshow(cv2.cvtColor(result['output'], cv2.COLOR_BGR2RGB))
        plt.title(f"{name}\nPSNR: {result['scores'][0]:.2f} dB")
    plt.tight_layout()
    plt.show()

def compare_images(img1, img2):
    """Calculate image quality metrics"""
    img1_gray = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    img2_gray = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
    return [
        psnr(img1, img2),
        mse(img1, img2),
        ssim(img1_gray, img2_gray, multichannel=True)
    ]

def attention_block(input_tensor):
    """Channel attention mechanism"""
    channels = input_tensor.shape[-1]
    gap = GlobalAvgPool2D()(input_tensor)
    fc1 = Dense(channels//8, activation='relu')(gap)
    fc2 = Dense(channels, activation='sigmoid')(fc1)
    attention = Reshape((1, 1, channels))(fc2)
    return Multiply()([input_tensor, attention])

def build_model(with_attention=True):
    """Build SRCNN with optional attention"""
    if with_attention:
        inputs = tf.keras.Input(shape=(None, None, 1))
        x = Conv2D(128, (9,9), activation='relu', padding='valid')(inputs)
        x = attention_block(x)
        x = Conv2D(64, (3,3), activation='relu', padding='same')(x)
        outputs = Conv2D(1, (5,5), activation='linear', padding='valid')(x)
        model = Model(inputs, outputs)
    else:
        model = Sequential([
            Conv2D(128, (9,9), activation='relu', padding='valid', input_shape=(None,None,1)),
            Conv2D(64, (3,3), activation='relu', padding='same'),
            Conv2D(1, (5,5), activation='linear', padding='valid')
        ])
    
    model.compile(optimizer=Adam(learning_rate=0.0003),
                 loss='mean_squared_error',
                 metrics=['mean_squared_error'])
    return model

def modcrop(img, scale):
    """Crop image to make dimensions divisible by scale"""
    h, w = img.shape[:2]
    h = h - h % scale
    w = w - w % scale
    return img[:h, :w]

def shave(image, border):
    """Remove border pixels"""
    return image[border:-border, border:-border]

def predict(image_path, use_attention=False):
    """Super-resolve an image with error handling"""
    try:
        srcnn = build_model(with_attention=use_attention)
        weights_path = '3051crop_weight_200.h5'
        srcnn.load_weights(weights_path)
        
        degraded = cv2.imread(image_path)
        ref_path = os.path.join('source', os.path.basename(image_path))
        ref = cv2.imread(ref_path)
        
        ref = modcrop(ref, 3)
        degraded = modcrop(degraded, 3)
        
        ycrcb = cv2.cvtColor(degraded, cv2.COLOR_BGR2YCrCb)
        Y = np.zeros((1, *ycrcb.shape[:2], 1), dtype=np.float32)
        Y[0, :, :, 0] = ycrcb[:, :, 0].astype(np.float32) / 255.0
        
        print(f"\nProcessing {os.path.basename(image_path)}")
        pre = srcnn.predict(Y, verbose=1)[0] * 255
        pre = np.clip(pre, 0, 255).astype(np.uint8)
        
        temp = shave(ycrcb, 6)
        temp[:, :, 0] = pre[:, :, 0]
        output = cv2.cvtColor(temp, cv2.COLOR_YCrCb2BGR)
        
        ref = shave(ref, 6)
        degraded = shave(degraded, 6)
        
        return ref, degraded, output, [
            compare_images(degraded, ref),
            compare_images(output, ref)
        ]
        
    except Exception as e:
        print(f"Error processing {image_path}: {str(e)}")
        return None, None, None, None

# Main processing loop
os.makedirs('output', exist_ok=True)

all_scores = []
image_names = []

for file in sorted(f for f in os.listdir('images') if f.lower().endswith('.bmp')):
    try:
        ref, degraded, output, scores = predict(os.path.join('images', file), use_attention=False)
        
        if ref is None or degraded is None or output is None:
            continue
            
        # Store results for final report
        all_scores.append(scores)
        image_names.append(os.path.splitext(file)[0])
            
        # Display metrics
        print("\n{:<20} {:>10} {:>12} {:>12}".format('', 'PSNR (dB)', 'MSE', 'SSIM'))
        print("-" * 50)
        print("{:<20} {:>10.2f} {:>12.2f} {:>12.4f}".format('Degraded Image:', *scores[0]))
        print("{:<20} {:>10.2f} {:>12.2f} {:>12.4f}".format('Reconstructed Image:', *scores[1]))
        
        # Interactive comparison for the first image
        if file == sorted(f for f in os.listdir('images') if f.lower().endswith('.bmp'))[0]:
            interactive_comparison(ref, degraded, output)
        
        # Architecture comparison for the first image
        if file == sorted(f for f in os.listdir('images') if f.lower().endswith('.bmp'))[0]:
            compare_architectures(os.path.join('images', file))
        
        # Save output
        base_name = os.path.splitext(file)[0]
        out_path = os.path.join('output', f"{base_name}_output.png")
        cv2.imwrite(out_path, cv2.cvtColor(output, cv2.COLOR_BGR2RGB))
        print(f"Saved to: {out_path}")
        
    except Exception as e:
        print(f"Error processing {file}: {str(e)}")

# Generate final quality report
if all_scores:
    generate_quality_report(image_names, all_scores)
    print("\nGenerated comprehensive quality report in output/quality_report.png")

/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)



Processing baboon.bmp
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 192ms/step

                      PSNR (dB)          MSE         SSIM
--------------------------------------------------
Degraded Image:           22.29       384.10       0.6734
Reconstructed Image:      23.04       323.18       0.7675


interactive(children=(IntSlider(value=5, description='alpha', max=10), Output()), _dom_classes=('widget-intera…


Processing baboon.bmp
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 186ms/step
Error processing baboon.bmp: Layer count mismatch when loading weights from file. Model expected 5 layers, found 3 saved layers.

Processing baby_GT.bmp
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 186ms/step

                      PSNR (dB)          MSE         SSIM
--------------------------------------------------
Degraded Image:           34.26        24.38       0.9396
Reconstructed Image:      36.27        15.36       0.9613
Saved to: output/baby_GT_output.png

Processing barbara.bmp
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 282ms/step

                      PSNR (dB)          MSE         SSIM
--------------------------------------------------
Degraded Image:           25.89       167.51       0.8183
Reconstructed Image:      26.58       142.90       0.8684
Saved to: output/barbara_output.png

Processing bird_GT.bmp
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step

                      PSNR (dB)          MSE         SSIM
--------------------------------------

In [ ]:
# # Main processing loop
# os.makedirs('output', exist_ok=True)

# for file in sorted(f for f in os.listdir('images') if f.lower().endswith('.bmp')):
#     try:
#         ref, degraded, output, scores = predict(os.path.join('images', file), use_attention=False)
        
#         if ref is None or degraded is None or output is None:
#             continue
            
#         # Display metrics in clean table format
#         print("\n{:<20} {:>10} {:>12} {:>12}".format('', 'PSNR (dB)', 'MSE', 'SSIM'))
#         print("-" * 50)
#         print("{:<20} {:>10.2f} {:>12.2f} {:>12.4f}".format('Degraded Image:', *scores[0]))
#         print("{:<20} {:>10.2f} {:>12.2f} {:>12.4f}".format('Reconstructed Image:', *scores[1]))
        
#         # Display images
#         fig, axs = plt.subplots(1, 3, figsize=(15, 5))
#         titles = ['Original', 'Degraded', 'SRCNN Output']
#         images = [ref, degraded, output]
        
#         for ax, img, title in zip(axs, images, titles):
#             ax.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
#             ax.set_title(title)
#             ax.set_xticks([])
#             ax.set_yticks([])
        
#         plt.tight_layout()
#         plt.show()
        
#         # Save output
#         base_name = os.path.splitext(file)[0]
#         out_path = os.path.join('output', f"{base_name}_output.png")
#         plt.savefig(out_path, bbox_inches='tight', dpi=150)
#         plt.close()
#         print(f"Saved to: {out_path}")
        
#     except Exception as e:
#         print(f"Error processing {file}: {str(e)}")
#         plt.close()

In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, GlobalAvgPool2D, Dense, Multiply, Reshape
from tensorflow.keras.optimizers import Adam
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import mean_squared_error as mse
from skimage.metrics import structural_similarity as ssim
from ipywidgets import interact, IntSlider
import pandas as pd
import seaborn as sns

def compare_images(img1, img2):
    """Calculate image quality metrics"""
    img1_gray = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    img2_gray = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
    return [
        psnr(img1, img2),
        mse(img1, img2),
        ssim(img1_gray, img2_gray, multichannel=True)
    ]

def attention_block(input_tensor):
    """Channel attention mechanism"""
    channels = input_tensor.shape[-1]
    gap = GlobalAvgPool2D()(input_tensor)
    fc1 = Dense(channels//8, activation='relu')(gap)
    fc2 = Dense(channels, activation='sigmoid')(fc1)
    attention = Reshape((1, 1, channels))(fc2)
    return Multiply()([input_tensor, attention])

def build_model(with_attention=False):  # Default changed to False
    """Build SRCNN with optional attention"""
    if with_attention:
        inputs = tf.keras.Input(shape=(None, None, 1))
        x = Conv2D(128, (9,9), activation='relu', padding='valid')(inputs)
        x = attention_block(x)
        x = Conv2D(64, (3,3), activation='relu', padding='same')(x)
        outputs = Conv2D(1, (5,5), activation='linear', padding='valid')(x)
        model = Model(inputs, outputs)
    else:
        model = Sequential([
            Conv2D(128, (9,9), activation='relu', padding='valid', input_shape=(None, None, 1)),
            Conv2D(64, (3,3), activation='relu', padding='same'),
            Conv2D(1, (5,5), activation='linear', padding='valid')
        ])
    
    model.compile(optimizer=Adam(learning_rate=0.0003),
                loss='mean_squared_error',
                metrics=['mean_squared_error'])
    return model

def modcrop(img, scale):
    """Crop image to make dimensions divisible by scale"""
    h, w = img.shape[:2]
    h = h - h % scale
    w = w - w % scale
    return img[:h, :w]

def shave(image, border):
    """Remove border pixels"""
    return image[border:-border, border:-border]

def predict(image_path, use_attention=False):
    """Super-resolve an image with error handling"""
    try:
        srcnn = build_model(with_attention=use_attention)
        weights_path = '3051crop_weight_200.h5'
        
        if not os.path.exists(weights_path):
            raise FileNotFoundError(f"Weights file {weights_path} not found")
        
        # Load weights with architecture mismatch handling
        try:
            srcnn.load_weights(weights_path)
        except ValueError as e:
            if "Layer count mismatch" in str(e):
                print("\n⚠️ Using basic model architecture to match weights file")
                srcnn = build_model(with_attention=False)
                srcnn.load_weights(weights_path)
            else:
                raise
                
        degraded = cv2.imread(image_path)
        if degraded is None:
            raise ValueError(f"Could not read image {image_path}")
            
        ref_path = os.path.join('source', os.path.basename(image_path))
        ref = cv2.imread(ref_path)
        if ref is None:
            raise ValueError(f"Could not read reference image {ref_path}")
        
        ref = modcrop(ref, 3)
        degraded = modcrop(degraded, 3)
        
        ycrcb = cv2.cvtColor(degraded, cv2.COLOR_BGR2YCrCb)
        Y = np.zeros((1, *ycrcb.shape[:2], 1), dtype=np.float32)
        Y[0, :, :, 0] = ycrcb[:, :, 0].astype(np.float32) / 255.0
        
        print(f"\nProcessing {os.path.basename(image_path)}")
        pre = srcnn.predict(Y, verbose=1)[0] * 255
        pre = np.clip(pre, 0, 255).astype(np.uint8)
        
        temp = shave(ycrcb, 6)
        temp[:, :, 0] = pre[:, :, 0]
        output = cv2.cvtColor(temp, cv2.COLOR_YCrCb2BGR)
        
        ref = shave(ref, 6)
        degraded = shave(degraded, 6)
        
        return ref, degraded, output, [
            compare_images(degraded, ref),
            compare_images(output, ref)
        ]
        
    except Exception as e:
        print(f"Error processing {image_path}: {str(e)}")
        return None, None, None, None

def interactive_comparison(original, degraded, enhanced):
    """Create interactive image comparison"""
    def display_images(alpha=0.5):
        plt.figure(figsize=(15, 5))
        plt.subplot(131)
        plt.imshow(cv2.cvtColor(original, cv2.COLOR_BGR2RGB))
        plt.title('Original')
        plt.axis('off')
        
        plt.subplot(132)
        plt.imshow(cv2.cvtColor(degraded, cv2.COLOR_BGR2RGB))
        plt.title('Degraded')
        plt.axis('off')
        
        plt.subplot(133)
        blended = cv2.addWeighted(degraded, 1-alpha, enhanced, alpha, 0)
        plt.imshow(cv2.cvtColor(blended, cv2.COLOR_BGR2RGB))
        plt.title(f'Enhanced (Blend: {alpha:.1f})')
        plt.axis('off')
        
        plt.tight_layout()
        plt.show()
    
    interact(display_images, alpha=IntSlider(min=0, max=10, step=1, value=5))

def generate_quality_report(image_names, all_scores):
    """Generate comprehensive quality report"""
    metrics_df = pd.DataFrame({
        'Image': image_names,
        'PSNR_Degraded': [s[0][0] for s in all_scores],
        'PSNR_Enhanced': [s[1][0] for s in all_scores],
        'MSE_Degraded': [s[0][1] for s in all_scores],
        'MSE_Enhanced': [s[1][1] for s in all_scores],
        'SSIM_Degraded': [s[0][2] for s in all_scores],
        'SSIM_Enhanced': [s[1][2] for s in all_scores]
    })
    
    metrics_df['PSNR_Improvement'] = metrics_df['PSNR_Enhanced'] - metrics_df['PSNR_Degraded']
    metrics_df['MSE_Reduction'] = metrics_df['MSE_Degraded'] - metrics_df['MSE_Enhanced']
    metrics_df['SSIM_Improvement'] = metrics_df['SSIM_Enhanced'] - metrics_df['SSIM_Degraded']
    
    metrics_df.to_csv('output/quality_metrics.csv', index=False)
    
    plt.figure(figsize=(15, 8))
    plt.subplot(231)
    sns.barplot(x='Image', y='PSNR_Improvement', data=metrics_df)
    plt.title('PSNR Improvement')
    plt.xticks(rotation=45)
    
    plt.subplot(232)
    sns.barplot(x='Image', y='MSE_Reduction', data=metrics_df)
    plt.title('MSE Reduction')
    plt.xticks(rotation=45)
    
    plt.subplot(233)
    sns.barplot(x='Image', y='SSIM_Improvement', data=metrics_df)
    plt.title('SSIM Improvement')
    plt.xticks(rotation=45)
    
    plt.tight_layout()
    plt.savefig('output/quality_report.png', dpi=300, bbox_inches='tight')
    plt.close()
    
    return metrics_df

# Main processing loop
os.makedirs('output', exist_ok=True)
all_scores = []
image_names = []

for file in sorted(f for f in os.listdir('images') if f.lower().endswith('.bmp')):
    try:
        ref, degraded, output, scores = predict(os.path.join('images', file), use_attention=False)
        
        if ref is None or degraded is None or output is None:
            continue
            
        all_scores.append(scores)
        image_names.append(os.path.splitext(file)[0])
            
        print("\n{:<20} {:>10} {:>12} {:>12}".format('', 'PSNR (dB)', 'MSE', 'SSIM'))
        print("-" * 50)
        print("{:<20} {:>10.2f} {:>12.2f} {:>12.4f}".format('Degraded Image:', *scores[0]))
        print("{:<20} {:>10.2f} {:>12.2f} {:>12.4f}".format('Reconstructed Image:', *scores[1]))
        
        # Show interactive comparison for first image only
        # if file == sorted(f for f in os.listdir('images') if f.lower().endswith('.bmp'))[0]:
        interactive_comparison(ref, degraded, output)
        
        # Save output images
        base_name = os.path.splitext(file)[0]
        cv2.imwrite(f'output/{base_name}_original.png', cv2.cvtColor(ref, cv2.COLOR_BGR2RGB))
        cv2.imwrite(f'output/{base_name}_degraded.png', cv2.cvtColor(degraded, cv2.COLOR_BGR2RGB))
        cv2.imwrite(f'output/{base_name}_reconstructed.png', cv2.cvtColor(output, cv2.COLOR_BGR2RGB))
        print(f"\nSaved results to output/{base_name}_*.png")
        
    except Exception as e:
        print(f"❌ Error processing {file}: {str(e)}")

# Generate final report if we processed any images
if all_scores:
    report = generate_quality_report(image_names, all_scores)
    print("\n" + "="*60)
    print("SUMMARY REPORT")
    print("="*60)
    print(f"Processed {len(image_names)} images successfully")
    print(f"Average PSNR Improvement: {report['PSNR_Improvement'].mean():.2f} dB")
    print(f"Average MSE Reduction: {report['MSE_Reduction'].mean():.2f}")
    print(f"Average SSIM Improvement: {report['SSIM_Improvement'].mean():.4f}")
    print("\nFull metrics saved to:")
    print("- output/quality_metrics.csv")
    print("- output/quality_report.png")

/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)



Processing baboon.bmp
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 260ms/step

                      PSNR (dB)          MSE         SSIM
--------------------------------------------------
Degraded Image:           22.29       384.10       0.6734
Reconstructed Image:      23.04       323.18       0.7675


interactive(children=(IntSlider(value=5, description='alpha', max=10), Output()), _dom_classes=('widget-intera…


Saved results to output/baboon_*.png

Processing baby_GT.bmp
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 188ms/step

                      PSNR (dB)          MSE         SSIM
--------------------------------------------------
Degraded Image:           34.26        24.38       0.9396
Reconstructed Image:      36.27        15.36       0.9613

Saved results to output/baby_GT_*.png

Processing barbara.bmp
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 271ms/step

                      PSNR (dB)          MSE         SSIM
--------------------------------------------------
Degraded Image:           25.89       167.51       0.8183
Reconstructed Image:      26.58       142.90       0.8684

Saved results to output/barbara_*.png

Processing bird_GT.bmp
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step

                      PSNR (dB)          MSE         SSIM
--------------------------------------------------
Degraded Image:           32.98        32.74       0.9593
Reconstructed Image:      36.56        14.37       0.9846

Saved results to 

In [1]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, GlobalAvgPool2D, Dense, Multiply, Reshape
from tensorflow.keras.optimizers import Adam
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import mean_squared_error as mse
from skimage.metrics import structural_similarity as ssim
from ipywidgets import interact, IntSlider
import pandas as pd
import seaborn as sns

def compare_images(img1, img2):
    """Calculate image quality metrics"""
    img1_gray = cv2.cvtColor(img1, cv2.COLOR_RGB2GRAY)  # Use RGB instead of BGR
    img2_gray = cv2.cvtColor(img2, cv2.COLOR_RGB2GRAY)
    return [
        psnr(img1, img2),
        mse(img1, img2),
        ssim(img1_gray, img2_gray, multichannel=True)
    ]

def attention_block(input_tensor):
    """Channel attention mechanism"""
    channels = input_tensor.shape[-1]
    gap = GlobalAvgPool2D()(input_tensor)
    fc1 = Dense(channels//8, activation='relu')(gap)
    fc2 = Dense(channels, activation='sigmoid')(fc1)
    attention = Reshape((1, 1, channels))(fc2)
    return Multiply()([input_tensor, attention])

def build_model(with_attention=False):
    """Build SRCNN with optional attention"""
    if with_attention:
        inputs = tf.keras.Input(shape=(None, None, 1))
        x = Conv2D(128, (9,9), activation='relu', padding='valid')(inputs)
        x = attention_block(x)
        x = Conv2D(64, (3,3), activation='relu', padding='same')(x)
        outputs = Conv2D(1, (5,5), activation='linear', padding='valid')(x)
        model = Model(inputs, outputs)
    else:
        model = Sequential([
            Conv2D(128, (9,9), activation='relu', padding='valid', input_shape=(None, None, 1)),
            Conv2D(64, (3,3), activation='relu', padding='same'),
            Conv2D(1, (5,5), activation='linear', padding='valid')
        ])
    
    model.compile(optimizer=Adam(learning_rate=0.0003),
                  loss='mean_squared_error',
                  metrics=['mean_squared_error'])
    return model

def modcrop(img, scale):
    """Crop image to make dimensions divisible by scale"""
    h, w = img.shape[:2]
    h = h - h % scale
    w = w - w % scale
    return img[:h, :w]

def shave(image, border):
    """Remove border pixels"""
    return image[border:-border, border:-border]

def predict(image_path, use_attention=False):
    """Super-resolve an image with error handling"""
    try:
        srcnn = build_model(with_attention=use_attention)
        weights_path = '3051crop_weight_200.h5'
        
        if not os.path.exists(weights_path):
            raise FileNotFoundError(f"Weights file {weights_path} not found")
        
        # Load weights with architecture mismatch handling
        try:
            srcnn.load_weights(weights_path)
        except ValueError as e:
            if "Layer count mismatch" in str(e):
                print("\n⚠️ Using basic model architecture to match weights file")
                srcnn = build_model(with_attention=False)
                srcnn.load_weights(weights_path)
            else:
                raise
                
        degraded = cv2.imread(image_path)[..., ::-1]  # Convert BGR to RGB
        if degraded is None:
            raise ValueError(f"Could not read image {image_path}")
            
        ref_path = os.path.join('source', os.path.basename(image_path))
        ref = cv2.imread(ref_path)[..., ::-1]  # Convert BGR to RGB
        if ref is None:
            raise ValueError(f"Could not read reference image {ref_path}")
        
        ref = modcrop(ref, 3)
        degraded = modcrop(degraded, 3)
        
        ycrcb = cv2.cvtColor(degraded, cv2.COLOR_RGB2YCrCb)
        Y = np.zeros((1, *ycrcb.shape[:2], 1), dtype=np.float32)
        Y[0, :, :, 0] = ycrcb[:, :, 0].astype(np.float32) / 255.0
        
        print(f"\nProcessing {os.path.basename(image_path)}")
        pre = srcnn.predict(Y, verbose=1)[0] * 255
        pre = np.clip(pre, 0, 255).astype(np.uint8)
        
        temp = shave(ycrcb, 6)
        temp[:, :, 0] = pre[:, :, 0]
        output = cv2.cvtColor(temp, cv2.COLOR_YCrCb2RGB)
        
        ref = shave(ref, 6)
        degraded = shave(degraded, 6)
        
        return ref, degraded, output, [
            compare_images(degraded, ref),
            compare_images(output, ref)
        ]
        
    except Exception as e:
        print(f"Error processing {image_path}: {str(e)}")
        return None, None, None, None

def interactive_comparison(original, degraded, enhanced):
    """Create interactive image comparison"""
    def display_images(alpha=0.5):
        plt.figure(figsize=(15, 5))
        plt.subplot(131)
        plt.imshow(original)  # No need to convert, it's already RGB
        plt.title('Original')
        plt.axis('off')
        
        plt.subplot(132)
        plt.imshow(degraded)
        plt.title('Degraded')
        plt.axis('off')
        
        plt.subplot(133)
        blended = cv2.addWeighted(degraded, 1-alpha, enhanced, alpha, 0)
        plt.imshow(blended)
        plt.title(f'Enhanced (Blend: {alpha:.1f})')
        plt.axis('off')
        
        plt.tight_layout()
        plt.show()
    
    interact(display_images, alpha=IntSlider(min=0, max=10, step=1, value=5))

# Main processing loop
os.makedirs('output', exist_ok=True)
all_scores = []
image_names = []

for file in sorted(f for f in os.listdir('images') if f.lower().endswith('.bmp')):
    try:
        ref, degraded, output, scores = predict(os.path.join('images', file), use_attention=False)
        
        if ref is None or degraded is None or output is None:
            continue
            
        all_scores.append(scores)
        image_names.append(os.path.splitext(file)[0])
            
        print("\n{:<20} {:>10} {:>12} {:>12}".format('', 'PSNR (dB)', 'MSE', 'SSIM'))
        print("-" * 50)
        print("{:<20} {:>10.2f} {:>12.2f} {:>12.4f}".format('Degraded Image:', *scores[0]))
        print("{:<20} {:>10.2f} {:>12.2f} {:>12.4f}".format('Reconstructed Image:', *scores[1]))
        
        interactive_comparison(ref, degraded, output)
        
        # Save output images correctly
        base_name = os.path.splitext(file)[0]
        cv2.imwrite(f'output/{base_name}_original.png', ref)
        cv2.imwrite(f'output/{base_name}_degraded.png', degraded)
        cv2.imwrite(f'output/{base_name}_reconstructed.png', output)
        print(f"\nSaved results to output/{base_name}_*.png")
        
    except Exception as e:
        print(f"❌ Error processing {file}: {str(e)}")


/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)



Processing baboon.bmp
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 227ms/step

                      PSNR (dB)          MSE         SSIM
--------------------------------------------------
Degraded Image:           22.29       384.10       0.6734
Reconstructed Image:      23.04       323.18       0.7675


interactive(children=(IntSlider(value=5, description='alpha', max=10), Output()), _dom_classes=('widget-intera…


Saved results to output/baboon_*.png

Processing baby_GT.bmp
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 205ms/step

                      PSNR (dB)          MSE         SSIM
--------------------------------------------------
Degraded Image:           34.26        24.38       0.9396
Reconstructed Image:      36.27        15.36       0.9613


interactive(children=(IntSlider(value=5, description='alpha', max=10), Output()), _dom_classes=('widget-intera…

...


Saved results to output/ppt3_*.png

Processing woman_GT.bmp
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step

                      PSNR (dB)          MSE         SSIM
--------------------------------------------------
Degraded Image:           29.37        75.11       0.9349
Reconstructed Image:      33.63        28.18       0.9677


interactive(children=(IntSlider(value=5, description='alpha', max=10), Output()), _dom_classes=('widget-intera…


Saved results to output/woman_GT_*.png

Processing zebra.bmp
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 155ms/step

                      PSNR (dB)          MSE         SSIM
--------------------------------------------------
Degraded Image:           27.78       108.36       0.8896
Reconstructed Image:      30.41        59.16       0.9324


interactive(children=(IntSlider(value=5, description='alpha', max=10), Output()), _dom_classes=('widget-intera…


Saved results to output/zebra_*.png


In [2]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, GlobalAvgPool2D, Dense, Multiply, Reshape
from tensorflow.keras.optimizers import Adam
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import mean_squared_error as mse
from skimage.metrics import structural_similarity as ssim
from ipywidgets import interact, IntSlider
import pandas as pd
import seaborn as sns

def compare_images(img1, img2):
    """Calculate image quality metrics"""
    img1_gray = cv2.cvtColor(img1, cv2.COLOR_RGB2GRAY)  # Use RGB instead of BGR
    img2_gray = cv2.cvtColor(img2, cv2.COLOR_RGB2GRAY)
    return [
        psnr(img1, img2),
        mse(img1, img2),
        ssim(img1_gray, img2_gray, multichannel=True)
    ]

def attention_block(input_tensor):
    """Channel attention mechanism"""
    channels = input_tensor.shape[-1]
    gap = GlobalAvgPool2D()(input_tensor)
    fc1 = Dense(channels//8, activation='relu')(gap)
    fc2 = Dense(channels, activation='sigmoid')(fc1)
    attention = Reshape((1, 1, channels))(fc2)
    return Multiply()([input_tensor, attention])

def build_model(with_attention=True):
    """Build SRCNN with optional attention"""
    if with_attention:
        inputs = tf.keras.Input(shape=(None, None, 1))
        x = Conv2D(128, (9,9), activation='relu', padding='valid')(inputs)
        x = attention_block(x)
        x = Conv2D(64, (3,3), activation='relu', padding='same')(x)
        outputs = Conv2D(1, (5,5), activation='linear', padding='valid')(x)
        model = Model(inputs, outputs)
    else:
        model = Sequential([
            Conv2D(128, (9,9), activation='relu', padding='valid', input_shape=(None, None, 1)),
            Conv2D(64, (3,3), activation='relu', padding='same'),
            Conv2D(1, (5,5), activation='linear', padding='valid')
        ])
    
    model.compile(optimizer=Adam(learning_rate=0.0003),
                  loss='mean_squared_error',
                  metrics=['mean_squared_error'])
    return model

def modcrop(img, scale):
    """Crop image to make dimensions divisible by scale"""
    h, w = img.shape[:2]
    h = h - h % scale
    w = w - w % scale
    return img[:h, :w]

def shave(image, border):
    """Remove border pixels"""
    return image[border:-border, border:-border]

def predict(image_path, use_attention=True):
    """Super-resolve an image with error handling"""
    try:
        srcnn = build_model(with_attention=use_attention)
        weights_path = '3051crop_weight_200.h5'
        
        if not os.path.exists(weights_path):
            raise FileNotFoundError(f"Weights file {weights_path} not found")
        
        # Load weights with architecture mismatch handling
        try:
            srcnn.load_weights(weights_path)
        except ValueError as e:
            if "Layer count mismatch" in str(e):
                print("\n⚠️ Using basic model architecture to match weights file")
                srcnn = build_model(with_attention=False)
                srcnn.load_weights(weights_path)
            else:
                raise
                
        degraded = cv2.imread(image_path)[..., ::-1]  # Convert BGR to RGB
        if degraded is None:
            raise ValueError(f"Could not read image {image_path}")
            
        ref_path = os.path.join('source', os.path.basename(image_path))
        ref = cv2.imread(ref_path)[..., ::-1]  # Convert BGR to RGB
        if ref is None:
            raise ValueError(f"Could not read reference image {ref_path}")
        
        ref = modcrop(ref, 3)
        degraded = modcrop(degraded, 3)
        
        ycrcb = cv2.cvtColor(degraded, cv2.COLOR_RGB2YCrCb)
        Y = np.zeros((1, *ycrcb.shape[:2], 1), dtype=np.float32)
        Y[0, :, :, 0] = ycrcb[:, :, 0].astype(np.float32) / 255.0
        
        print(f"\nProcessing {os.path.basename(image_path)}")
        pre = srcnn.predict(Y, verbose=1)[0] * 255
        pre = np.clip(pre, 0, 255).astype(np.uint8)
        
        temp = shave(ycrcb, 6)
        temp[:, :, 0] = pre[:, :, 0]
        output = cv2.cvtColor(temp, cv2.COLOR_YCrCb2RGB)
        
        ref = shave(ref, 6)
        degraded = shave(degraded, 6)
        
        return ref, degraded, output, [
            compare_images(degraded, ref),
            compare_images(output, ref)
        ]
        
    except Exception as e:
        print(f"Error processing {image_path}: {str(e)}")
        return None, None, None, None

def interactive_comparison(original, degraded, enhanced):
    """Create interactive image comparison"""
    def display_images(alpha=0.5):
        plt.figure(figsize=(15, 5))
        plt.subplot(131)
        plt.imshow(original)  # No need to convert, it's already RGB
        plt.title('Original')
        plt.axis('off')
        
        plt.subplot(132)
        plt.imshow(degraded)
        plt.title('Degraded')
        plt.axis('off')
        
        plt.subplot(133)
        blended = cv2.addWeighted(degraded, 1-alpha, enhanced, alpha, 0)
        plt.imshow(blended)
        plt.title(f'Enhanced (Blend: {alpha:.1f})')
        plt.axis('off')
        
        plt.tight_layout()
        plt.show()
    
    interact(display_images, alpha=IntSlider(min=0, max=10, step=1, value=5))

# Main processing loop
os.makedirs('output', exist_ok=True)
all_scores = []
image_names = []

for file in sorted(f for f in os.listdir('images') if f.lower().endswith('.bmp')):
    try:
        ref, degraded, output, scores = predict(os.path.join('images', file), use_attention=False)
        
        if ref is None or degraded is None or output is None:
            continue
            
        all_scores.append(scores)
        image_names.append(os.path.splitext(file)[0])
            
        print("\n{:<20} {:>10} {:>12} {:>12}".format('', 'PSNR (dB)', 'MSE', 'SSIM'))
        print("-" * 50)
        print("{:<20} {:>10.2f} {:>12.2f} {:>12.4f}".format('Degraded Image:', *scores[0]))
        print("{:<20} {:>10.2f} {:>12.2f} {:>12.4f}".format('Reconstructed Image:', *scores[1]))
        
        interactive_comparison(ref, degraded, output)
        
        # Save output images correctly
        base_name = os.path.splitext(file)[0]
        cv2.imwrite(f'output/{base_name}_original.png', ref)
        cv2.imwrite(f'output/{base_name}_degraded.png', degraded)
        cv2.imwrite(f'output/{base_name}_reconstructed.png', output)
        print(f"\nSaved results to output/{base_name}_*.png")
        
    except Exception as e:
        print(f"❌ Error processing {file}: {str(e)}")



Processing baboon.bmp
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 169ms/step

                      PSNR (dB)          MSE         SSIM
--------------------------------------------------
Degraded Image:           22.29       384.10       0.6734
Reconstructed Image:      23.04       323.18       0.7675


interactive(children=(IntSlider(value=5, description='alpha', max=10), Output()), _dom_classes=('widget-intera…


Saved results to output/baboon_*.png

Processing baby_GT.bmp
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 198ms/step

                      PSNR (dB)          MSE         SSIM
--------------------------------------------------
Degraded Image:           34.26        24.38       0.9396
Reconstructed Image:      36.27        15.36       0.9613


interactive(children=(IntSlider(value=5, description='alpha', max=10), Output()), _dom_classes=('widget-intera…


Saved results to output/baby_GT_*.png

Processing barbara.bmp
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 293ms/step

                      PSNR (dB)          MSE         SSIM
--------------------------------------------------
Degraded Image:           25.89       167.51       0.8183
Reconstructed Image:      26.58       142.90       0.8684


...


Saved results to output/ppt3_*.png

Processing woman_GT.bmp
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step

                      PSNR (dB)          MSE         SSIM
--------------------------------------------------
Degraded Image:           29.37        75.11       0.9349
Reconstructed Image:      33.63        28.18       0.9677


interactive(children=(IntSlider(value=5, description='alpha', max=10), Output()), _dom_classes=('widget-intera…


Saved results to output/woman_GT_*.png

Processing zebra.bmp
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 165ms/step

                      PSNR (dB)          MSE         SSIM
--------------------------------------------------
Degraded Image:           27.78       108.36       0.8896
Reconstructed Image:      30.41        59.16       0.9324


interactive(children=(IntSlider(value=5, description='alpha', max=10), Output()), _dom_classes=('widget-intera…


Saved results to output/zebra_*.png
